# VNPY Backtest Test

## Import Modules

In [17]:
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from vnpy.app.cta_strategy.base import BacktestingMode   #backtesting 
from datetime import datetime    #time
from vnpy.trader.constant import Interval, Exchange
# from vnpy.app.atr_rsi_strategy import AtrRsiStrategy

from vnpy.app.portfolio_strategy.backtesting import BacktestingEngine as PairBacktestEngine

import matplotlib
%matplotlib inline

## Create my Method

In [18]:
import re


def getCommission(symbol,
                  exchange,
                  cost,
                  multiplier,
                  qty,
                  Real=False,
                  direction=''):
    """計算個別部位的單邊交易成本"""
    try:
        if exchange == 'TFE':
            if re.match('MTX[0-9]*', symbol):  # 小台指期
                return 50 + (1 * 50 + cost * (2 / 100000) * multiplier
                             ) * qty if not Real else 50 + (cost *
                                                            (2 / 100000) *
                                                            multiplier) * qty
            elif re.match('TX[0-9]*',
                          symbol) or re.match('TE[0-9]*', symbol) or re.match(
                              'TF[0-9]*', symbol):  # 大台指期
                return 50 + (1 * 200 + cost * (2 / 100000) * multiplier
                             ) * qty if not Real else 50 + (cost *
                                                            (2 / 100000) *
                                                            multiplier) * qty
            elif re.match(r"TX[O0-9][0-9]*[A-Z][0-9]", symbol):  # 台指選
                return 30 + (getMinimumTickOpt(cost) + cost * 2 /
                             100000) * qty * multiplier if not Real else 30 + (
                                 cost * 2 / 100000) * qty * multiplier
            elif re.match(r"[A-Z][A-Z]F[0-9][0-9]", symbol):  # 個股期貨
                return 50 + (
                    getMinimumTick(cost) + cost *
                    (2 / 100000)) * qty * multiplier if not Real else 50 + (
                        cost * (2 / 100000)) * qty * multiplier
        elif exchange == 'TSE':
            tick = getMinimumTick(cost)
            commission = cost * (0.1425 / 100) * multiplier
            commission = 20 if commission < 20 else commission
            fee = cost * (0.3 / 100) * multiplier
            slide = tick * multiplier
            if direction == 'EXIT' or direction == 0:
                return (commission + fee +
                        slide) * qty if not Real else (commission + fee) * qty
            else:
                return (commission +
                        slide) * qty if not Real else commission * qty
    except Exception as e:
        print(e)


def getMinimumTickOpt(cost):
    if cost < 10:
        return 0.1
    elif cost < 50:
        return 0.5
    elif cost < 500:
        return 1
    elif cost < 1000:
        return 5
    else:
        return 10


def getMinimumTick(cost):
    if cost < 10:
        return 0.01
    elif cost < 50:
        return 0.05
    elif cost < 100:
        return 0.1
    elif cost < 500:
        return 0.5
    elif cost < 1000:
        return 1
    else:
        return 5

以上皆與turtle strategy相同原始成本設定

## setup bollChannelStrategy+CCI indicator


In [24]:
from vnpy.app.cta_strategy import (
    CtaTemplate,
    StopOrder,
    TickData,
    BarData,
    TradeData,
    OrderData,
    BarGenerator,
    ArrayManager,
)


class BollChannelStrategy(CtaTemplate):
    """"""

    author = "Xi Qi Capital"

    boll_window = 18
    boll_dev = 3.4
    cci_window = 10
    atr_window = 30
    sl_multiplier = 5.2     
    fixed_size = 1

    boll_up = 0
    boll_down = 0
    cci_value = 0
    atr_value = 0

    intra_trade_high = 0
    intra_trade_low = 0
    long_stop = 0
    short_stop = 0

    parameters = [
        "boll_window",
        "boll_dev",
        "cci_window",
        "atr_window",
        "sl_multiplier",
        "fixed_size"
    ]
    variables = [
        "boll_up",
        "boll_down",
        "cci_value",
        "atr_value",
        "intra_trade_high",
        "intra_trade_low",
        "long_stop",
        "short_stop"
    ]

    def __init__(self, cta_engine, strategy_name, vt_symbol, setting):
        """"""
        super().__init__(cta_engine, strategy_name, vt_symbol, setting)

        self.bg = BarGenerator(self.on_bar)
        self.am = ArrayManager()

    def on_init(self):
        """
        Callback when strategy is inited.
        """
        self.write_log("策略初始化")
        self.load_bar(10)

    def on_start(self):
        """
        Callback when strategy is started.
        """
        self.write_log("策略启动")

    def on_stop(self):
        """
        Callback when strategy is stopped.
        """
        self.write_log("策略停止")

    def on_tick(self, tick: TickData):
        """
        Callback of new tick data update.
        """
        self.bg.update_tick(tick)

    def on_bar(self, bar: BarData):
        """
        Callback of new bar data update.
        """
        self.bg.update_bar(bar)

    def on_bar(self, bar: BarData):
        """"""
        self.cancel_all()                  #確認沒有其他order單                                   

        am = self.am
        am.update_bar(bar)                 #arrary manager更新最新bar
        if not am.inited:
            return

        self.boll_up, self.boll_down = am.boll(self.boll_window, self.boll_dev)    #array manager內有布林通道function，輸入參數即可
        self.cci_value = am.cci(self.cci_window)                                   #計算commodity channel index
        self.atr_value = am.atr(self.atr_window)                                   #計算每日ATR

        if self.pos == 0:
            self.intra_trade_high = bar.high_price
            self.intra_trade_low = bar.low_price

            if self.cci_value > 0:
                self.buy(self.boll_up, self.fixed_size, True)                     #CCI>0，高於平均水準，以布林通道上緣做發散策略
            elif self.cci_value < 0:
                self.short(self.boll_down, self.fixed_size, True)                 #CCI<0,低於平均水準，以布林通道下原作發散策略

        elif self.pos > 0:                                                        #持有多方倉位
            self.intra_trade_high = max(self.intra_trade_high, bar.high_price)    #判斷日内交易的最高值
            self.intra_trade_low = bar.low_price

            self.long_stop = self.intra_trade_high - self.atr_value * self.sl_multiplier   #平倉條件: 最高價-(波動度*給定乘數)
            self.sell(self.long_stop, abs(self.pos), True)

        elif self.pos < 0:
            self.intra_trade_high = bar.high_price
            self.intra_trade_low = min(self.intra_trade_low, bar.low_price)

            self.short_stop = self.intra_trade_low + self.atr_value * self.sl_multiplier
            self.cover(self.short_stop, abs(self.pos), True)

        self.put_event()

    def on_order(self, order: OrderData):
        """
        Callback of new order data update.
        """
        pass

    def on_trade(self, trade: TradeData):
        """
        Callback of new trade data update.
        """
        self.put_event()

    def on_stop_order(self, stop_order: StopOrder):
        """
        Callback of stop order update.
        """
        pass


In [20]:
from functools import partial

In [25]:
# from vnpy.trader.constant import Interval, Exchange

tickers = '2330.TSE'
symbol, exchange = tickers.split('.')
get_commission = partial(getCommission, symbol=symbol, exchange=exchange)
engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol=tickers,
    interval=Interval.DAILY,
    start=datetime(2008, 1, 1),
    end=datetime(2020, 6, 30),
    rate=get_commission,
    slippage=getMinimumTick,
    size=1000,
    pricetick=getMinimumTick,
    capital=100_000,
    mode=BacktestingMode.BAR,
    collection_name='TSE')

boll_setting = {
    'boll_window' : 18,
    'boll_dev' : 2,
    'cci_window' : 10,
    'atr_window' : 30,
    'sl_multiplier' : 5.2,
    'fixed_size' : 1,
}
engine.add_strategy(BollChannelStrategy, boll_setting)
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()
engine.show_chart()

2020-11-23 11:45:56.146320	开始加载历史数据
2020-11-23 11:45:56.146320	加载进度： [1%]
2020-11-23 11:45:56.146320	加载进度： [1%]
2020-11-23 11:45:56.146320	加载进度： [2%]
2020-11-23 11:45:56.146320	加载进度： [3%]
2020-11-23 11:45:56.146320	加载进度： [3%]
2020-11-23 11:45:56.146320	加载进度： [4%]
2020-11-23 11:45:56.146320	加载进度： [5%]
2020-11-23 11:45:56.146320	加载进度： [5%]
2020-11-23 11:45:56.146320	加载进度： [6%]
2020-11-23 11:45:56.146320	加载进度： [7%]
2020-11-23 11:45:56.146320	加载进度： [7%]
2020-11-23 11:45:56.146320	加载进度： [8%]
2020-11-23 11:45:56.146320	加载进度： [9%]
2020-11-23 11:45:56.146320	加载进度： [9%]
2020-11-23 11:45:56.146320	加载进度： [10%]
2020-11-23 11:45:56.147336	加载进度：# [11%]
2020-11-23 11:45:56.147336	加载进度：# [11%]
2020-11-23 11:45:56.147336	加载进度：# [12%]
2020-11-23 11:45:56.147336	加载进度：# [12%]
2020-11-23 11:45:56.147336	加载进度：# [13%]
2020-11-23 11:45:56.147336	加载进度：# [14%]
2020-11-23 11:45:56.147336	加载进度：# [14%]
2020-11-23 11:45:56.147336	加载进度：# [15%]
2020-11-23 11:45:56.147336	加载进度：# [16%]
2020-11-23 11:45:56.147336	加载进度：#

C:\Users\j4a3m\anaconda3\lib\site-packages\vnpy\app\cta_strategy\backtesting.py:399: RuntimeWarning:

invalid value encountered in log

